# Analytical form factor
Dilute SANS samples have $Q$-ranges corresponding to their form factor. The above simulates a very naive scattering model in which $Q_y = Q$ is used. Next, proper radial scattering is considered.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from definitions import *
from scipy import integrate

In [ ]:
def form_factor(Q, R):
    return (3 * (np.sin(Q * R) - Q * R * np.cos(Q * R))/ (Q * R) ** 3)**2

Rs = [50e-9, 300e-9, 2000e-9]
Q = np.linspace(0, 0.015e10,100000)
# for R in Rs:
R = 100e-9
ff = form_factor(Q,R)
plt.plot(Q * 1e-10,ff)
# plt.xscale('log')
plt.yscale('log')
plt.xlabel(r'$Q$ [Å$^{-1}$]')
plt.ylabel(r'$P(Q)$')
plt.xlim((0.0,0.015))

plt.ylim((1e-7,1.1))
plt.axvline(0.1 / R * 1e-10, linestyle='--', color='red', label=r'$Q_\text{min} = 0.0001$ Å$^{-1}$')
plt.axvline(10 / R * 1e-10, linestyle='--', color='green', label=r'$Q_\text{max} = 0.01$ Å$^{-1}$')
plt.legend()
plt.grid()
plt.savefig("docs/analytical-P-log.eps",bbox_inches="tight", pad_inches=0, format='eps')

# 1D scattering model with $Q = Q_y$

In [ ]:
L_s = 1.8

def form_factor_y_naive(Q, R):
    Q = np.abs(Q)
    return (3 * (np.sin(Q * R) - Q * R * np.cos(Q * R))/ (Q * R) ** 3)**2

def Qy_to_y(Q_y, s_y, k):
    y = L_s * np.tan(2 * np.arcsin(Q_y / (2 * k)))
    return y + s_y

def y_to_Qy(y, s_y, k):
    Qy = 2 * k * np.sin(np.arctan((y - s_y) / L_s)/2)
    return Qy

def form_factor_y(y,s_y, k, R):
    Qy = y_to_Qy(y,s_y, k)
    return form_factor_y_naive(Qy,R)
    
def ff_area(R, k):
    y_lim = 0.025e-7 / R
    y_oversized = np.linspace(-y_lim,y_lim, 10000)
    dy = y_oversized[1] - y_oversized[0]
    Q = y_to_Qy(y_oversized,0,k)
    ff = form_factor_y_naive(Q,R)
    ff_area = integrate.trapezoid(ff) * dy
    # print(ff_area)
    return ff_area

R = 50e-9
Q_max = 0.015e10
y_lim = 0.025e-7 / R
y_oversized = np.linspace(-y_lim,y_lim, 10000)
# dQ = Q[1] - Q[0]
norm_map = {}
for lambda_0 in [4.321e-10, 8e-10]:
    k = 2 * np.pi / lambda_0
    Q = y_to_Qy(y_oversized,0,k)
    ff = form_factor_y_naive(Q,R)
    # plt.plot(Q * 1e-10,ff)
    ff_norm_factor = ff_area(R, k)
    ff_norm = ff / ff_norm_factor
    # norm_map[lambda_0] = ff_area
    # print(ff_area)
    # ff_norm = ff / ff_area
    y = Qy_to_y(Q, 0, k=k)
    plt.plot(y * 1e3,ff_norm, label=rf'$\lambda_0 = {lambda_0 * 1e10}Å$')
    # plt.yscale('log')
    plt.legend()
    plt.xlabel(r'$y$ [mm]')
    plt.ylabel(r'$P(y)$')

In [ ]:

# Q_max = 0.015e10
# Q = np.linspace(-Q_max, Q_max,100000)
h_d = detector_size
y = np.linspace(-h_d / 2, h_d/2, 4000)
# dQ = Q[1] - Q[0]
dy = y[1] - y[0]
print(R)
for (lambda_0, delta_limit) in [(4.321e-10, 70.71e-9), (8e-10,130.91)]:
    for R in [50e-9, 100e-9, 200e-9, 2000e-9]:
        k = 2 * np.pi / lambda_0
        N_sample_points = 4000
        s_y_range = np.linspace(-0.005,0.005,N_sample_points)
        I_t = np.zeros_like(y)
        norm_factor = ff_area(R, k)
        s_y_grid, y_grid = np.meshgrid(s_y_range, y, indexing='ij')
        ff = form_factor_y(y_grid, s_y_grid, k, R) / (N_sample_points * norm_factor)
        # Sum along the s_y dimension and scale by 0.01
        I_t = np.sum(ff, axis=0) * 0.01
        
        # The slightly more readable, non-vectorized form is this:
        #
        # for s_y in s_y_range:
        #     ff = form_factor_y(y, s_y, k, R) / (N_sample_points * norm_factor)
        #     # plt.plot(y, ff)
        #     # plt.plot(y[indices] * 1e3,ff[indices])
        #     I_t += ff * 0.01

        plt.plot(y * 1e3, I_t, label=rf'$R = {round(R * 1e9)}$nm')
        plt.xlabel(r'$y$ [mm]')
        plt.ylabel(r'$I(y)$')
    plt.legend()
    plt.xlim((-h_d/2 * 1e3,h_d/2 * 1e3))
    plt.grid()
    plt.savefig(f"docs/simplified-I-scattering-{lambda_0 * 1e10}.eps",bbox_inches="tight", pad_inches=0, format='eps')

    plt.show()
# y  = np.linspace(-10e-3, 10e-3)


# 2D scattering model
The above model simply equates $Q_y = Q$, whereas in fact $P(Q)$ says something about a radial distribution. In the actual detector, for a given $Q_y$ or $y$, all $Q_x$/$x$ will be integrated over the long detector element. This corresponds also to the $G_\text{exp}(\delta)$ expression which performs a cosine transform in one dimension and simply integrates in the other. This can be accounted for by consider the true radially symmetric scattering due to the sphere form factor $P(Q)$, which is done here.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

PLOT_3D = True

N_x = 1000
N_y = 1000

def form_factor_2D(x, s_x, y, s_y, k, R):
    Qx = y_to_Qy(x,s_x, k)
    Qy = y_to_Qy(y,s_y, k)
    Q = np.sqrt(Qx ** 2 + Qy ** 2)
    return form_factor(Q,R)

    
def ff_norm_factor_2D(R, k):
    x_lim = 0.025e-7 / R
    y_lim = x_lim
    x = np.linspace(-x_lim,x_lim, N_x)
    y = np.linspace(-y_lim,y_lim, N_y)
    X, Y = np.meshgrid(x, y)
    Z = form_factor_2D(X, 0, Y, 0, k, R)
    dx = x[1] - x[0]
    dy = y[1] - y[0]
    Z_y = integrate.trapezoid(Z, axis = 1, dx = dx)
    scale_factor = integrate.trapezoid(Z_y, dx = dy)
    return scale_factor

def plot_3D_intensity(X, Y, I):
    if PLOT_3D:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        surf = ax.plot_surface(X * 1e3, Y * 1e3, I * 1e-6, cmap='viridis', edgecolor='none')
        cbar = fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10, pad=0.1)
        cbar.set_label(r'$I_{(s_x,s_y) = (0,0)}$ [a.u.]', fontsize=12)
        ax.set_xlabel(r'$x$ [mm]')
        ax.set_ylabel(r'$y$ [mm]')
        plt.savefig(f"docs/simplified-I-scattering-3D-plot-point-{lambda_0 * 1e10}.eps",bbox_inches="tight", pad_inches=0, format='eps')
        plt.show()
    else:
        x_lim, y_lim = np.min(X[0,:]), np.min(Y[0,:])
        plt.imshow(I, extent=[-x_lim, x_lim, -y_lim, y_lim], origin='lower', cmap='viridis')
        plt.colorbar(label='Function Value')
        plt.xlabel('x')
        plt.ylabel('y')
        plt.show()
for (lambda_0, delta_limit) in [(4.321e-10, 70.71e-9), (8e-10,130.91)]:
    k = 2 * np.pi / lambda_0
    R = 200e-9
    x_lim = h_d / 2
    y_lim = x_lim
    x = np.linspace(-x_lim,x_lim, N_x)
    y = np.linspace(-y_lim,y_lim, N_y)
    X, Y = np.meshgrid(x, y, indexing='ij')
    Z = form_factor_2D(X, 0, Y, 0, k, R)
    scale_factor = ff_norm_factor_2D(R,k)
    print(scale_factor)
    Z_unit = Z / scale_factor
    plot_3D_intensity(X,Y,Z_unit)

# Integrate intensity curves from full sample 2D intensity pattern

In [ ]:
h_d = detector_size
h_s = 0.01
N_x = 200
N_y = 200
N_sample_points = 100
dy = y[1] - y[0]
print(R)
I_t_map = {}
for (lambda_0, delta_limit) in [(4.321e-10, 70.71e-9), (8e-10,130.91)]:
    for R in [50e-9, 100e-9, 200e-9, 300e-9, 2000e-9]:
        k = 2 * np.pi / lambda_0
        x_lim = h_d / 2
        y_lim = x_lim
        x = np.linspace(-x_lim,x_lim, N_x)
        y = np.linspace(-y_lim,y_lim, N_y)
        X, Y = np.meshgrid(x, y, indexing='ij')
        scale_factor = ff_norm_factor_2D(R,k)
        s_x_range = np.linspace(-h_s/2,h_s/2,N_sample_points)
        s_y_range = np.linspace(-h_s/2,h_s/2,N_sample_points)
        I_t = np.zeros_like(X)
        for s_x in s_x_range:
            s_y_grid, X_grid, Y_grid = np.meshgrid(s_y_range, x, y, indexing='ij')
            ff = form_factor_2D(X_grid, s_x, Y_grid, s_y_grid, k, R) / (N_sample_points * N_sample_points * scale_factor)
            I_t += np.sum(ff, axis=0)
        I_t *= 0.01
        I_t_map[(lambda_0, R)] = I_t

In [ ]:
for (lambda_0, delta_limit) in [(4.321e-10, 70.71e-9), (8e-10,130.91)]:
    for R in [50e-9, 100e-9, 200e-9, 300e-9, 2000e-9]:
        x_lim = h_d / 2
        y_lim = x_lim
        y = np.linspace(-y_lim,y_lim, N_y)
        I_t = I_t_map[(lambda_0, R)]
        I_t_y = integrate.trapezoid(I_t, x=y, axis=1)
        plt.plot(y * 1e3, I_t_y, label=rf'$R = {round(R * 1e9)}$nm')
        plt.xlabel(r'$y$ [mm]')
        plt.ylabel(r'$I(y)$')
    plt.legend()
    plt.xlim((-h_d/2 * 1e3,h_d/2 * 1e3))
    plt.grid()
    plt.savefig(f"docs/simplified-I-scattering-{lambda_0 * 1e10}.eps",bbox_inches="tight", pad_inches=0, format='eps')
    plt.show()


In [ ]:
# import pickle
# with open('objects/I_t.pkl', 'wb') as file:
#     pickle.dump(I_t, file)

In [ ]:
import pickle
with open('objects/I_t.pkl', 'rb') as file:
    I_t = pickle.load(file)

In [ ]:
for (lambda_0, delta_limit) in [(4.321e-10, 70.71e-9), (8e-10,130.91)]:
    # for R in [50e-9, 100e-9, 200e-9, 300e-9, 2000e-9]:
    for R in [200e-9]:
        I_t = I_t_map[(lambda_0, R)]
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        surf = ax.plot_surface(X * 1e3, Y * 1e3, I_t, cmap='viridis', edgecolor='none')
        cbar = fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10, pad=0.1)
        cbar.set_label(r'$I$ [a.u.]', fontsize=12)
        ax.set_xlabel(r'$x$ [mm]')
        ax.set_ylabel(r'$y$ [mm]')
        plt.savefig(f"docs/simplified-I-scattering-3D-plot-{lambda_0 * 1e10}.eps",bbox_inches="tight", pad_inches=0, format='eps')

        plt.show()
        plt.imshow(I_t, extent=[-x_lim * 1e3, x_lim * 1e3, -y_lim * 1e3, y_lim * 1e3], origin='lower', cmap='viridis')
        plt.colorbar(label=r'$I$ [a.u.]')
        plt.xlabel(r'$x$ [mm]')
        plt.ylabel(r'$y$ [mm]')
        plt.show()
        plt.savefig(f"docs/simplified-I-scattering-2D-image-{lambda_0 * 1e10}.eps",bbox_inches="tight", pad_inches=0, format='eps')
